In [3]:
#imports
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [58]:
test = np.random.normal(size=(6,3))
#print(test)
#print(np.mean(test,axis=0))
#print(np.sum(test, axis=0))
#print(5**-1)
#print(test.shape[1])
#print(test[1], test[1,:])
#a = [[1,2],[3,4]]
#b = np.zeros(shape=(2,2))
#c = np.concatenate((a,b))
#print(c.shape)

words = test
males = words[:3]
females = words[3:]
# print(male.shape, female.shape)
result = double_hard_debias(words, males, females)

In [57]:
def double_hard_debias(words, males, females):
    """Double Hard Debias:
    
    words: word embeddings of some corpus
    males: set of most biased male words 
    females: set of most biased female words
    """
    
    #need: Word embeddings, top 500 Male biased words set Wm, top 500 Female biased words set Wf
    #1. for all word embeddings: decentralize all words
    mue = (len(words)**(-1)) * np.sum(words, axis=0)
    # print(mue)
    words_decen = np.zeros((words.shape))
    for index, embedding in enumerate(words):
        # print(index,":",embedding)
        words_decen[index] = embedding - mue
    
    #print("decentralized:",words_decen)
    #print("origin:",words)
        
    #2. for all decentralized embeddings: compute PCA
    #princ_comp = np.asarray(pca_tft(words_decen))
    #print("Principal Components:",princ_comp)
    pca = PCA().fit(words_decen)
    princ_comp = pca.components_

    #print("Sklearn PC:", pca.components_)

    evaluations = []

    #3. for all principal components:
    for pc in princ_comp:
        male_proj = np.zeros((males.shape))
        male_debias = np.zeros((males.shape))
        female_proj = np.zeros((females.shape))
        female_debias = np.zeros((females.shape))
   
        for index, male in enumerate(males):
        #male embedding = decentralized embedding - projected original (?) embedding into direction of PC
            male_proj[index] = (male - mue) - ((np.transpose(pc)*male)*pc)
            #with all new male embeddings: HardDebias
            male_debias[index] = hardDebias(male_proj[index])
        
        for index, female in enumerate(females):
        #female embedding = decentralized embedding - projected original (?) embedding into direction of PC
            female_proj[index] = (female - mue) - ((np.transpose(pc)*male)*pc)
            #with all new female embeddings: HardDebias
            female_debias[index] = hardDebias(female_proj[index])
    
        #for all HardDebiased embeddings: KMeansClustering (2)
        #for clustered embeddings: compute gender alignment accuracy
        #4. store evaluations for each principal components
        evaluations.append(align_acc(male_debias, female_debias))
    
    #5. evaluate which PC lead to most random cluster (evaluation smallest (close to 0.5), used second PC)
    best_eval = evaluations.index(np.min(evaluations))
    best_pc = princ_comp[best_eval]
    #print("Best PC:",best_pc,"with evaluation:",evaluations[best_eval])

    first_debias = np.zeros((words.shape))
    #6. for all decentralized embeddings: remove that PC-direction
    for index,word in enumerate(words_decen):
        first_debias[index] = word - ((np.transpose(best_pc)*words[index])*best_pc)
    
    #7. for all new embeddings: HardDebias
    double_debias = np.zeros((words.shape))
    for index,word in enumerate(first_debias):
        double_debias[index] = hardDebias(word)

    return double_debias


In [53]:
#Gender alignment accuracy/ Neighborhood Metric:
def align_acc(males, females):
    """bias measurement using KMeans Clustering
    
    takes female and male word's embeddings
    ground truth labels:
    0 = male,
    1 = female"""
    
    #need: k (=1000) most biased female and male word's embedding (cosine similarity embedding & gender direction),
    #1. assign ground truth gender labels: 0 = male, 1 = female
    #2. run KMeans on embeddings
    kmeans = KMeans(n_clusters=2).fit(np.concatenate((males, females)))
    split = males.shape[0]
    correct = 0
    #print(kmeans.labels_)
    #3. compute alignment score: cluster assignment vs ground truth gender label
    for i in range(np.concatenate((males, females)).shape[0]):
        if i < split and kmeans.labels_[i] == 0:
            correct+= 1
        elif i >= split and kmeans.labels_[i] == 1:
            correct += 1
    
    #4. alignment score = max(a, 1-a)
    alignment = 1/(2*2) * correct
    alignment = np.maximum(alignment, 1-alignment)
    
    return alignment 

In [50]:
def hardDebias(something):
    """Placeholder"""
    return something